## ScenarioJudger

 - Reads a file from S3 containing json compliance scenarios of the format:
```json
{
  "scenarios": [
    {
      "scenario-id": "scenario-id-1",
      "scenario-detail": "A new employee, Sarah Johnson, joins the IT department...",
      "is-compliant": false,
      "non-compliant-reason": "The scenario violates..." 
    },
    {
      "scenario-id": "scenario-id-2", 
      "scenario-detail": "TechCorp implements a comprehensive incident response procedure...",
      "is-compliant": true,
      "non-compliant-reason": "" 
    }
  ]
}
```
 - Evaluates the veracity each scenario-detail based on RAGed NIST-based policies in Bedrock knowledgebase, comparing its determination against "is-compliant" in the json.
 - When its determination differs, generates json records:
```json
{
  "scenarios": [
    {
      "scenario-id": "scenario-id-1",
      "scenario-detail": "A new employee, Sarah Johnson, joins the IT department...",
      "is-compliant": false,
      "non-compliant-reason": "The scenario violates...",
      "judged-compliant": true,
      "judged-compliant-reason": "Considered the rules AC...  and scenario is not in violation..."
      "llm-judge": "us.anthropic.claude-sonnet-4-20250514-v1:0",
      "judged-dtm":  
    },
    {
      "scenario-id": "scenario-id-2", 
      "scenario-detail": "TechCorp implements a comprehensive incident response procedure...",
      "is-compliant": true,
      "non-compliant-reason": "", 
      "judged-compliant": false,
      "judged-compliant-reason": "Scenario violates access control policy...",
      "llm-judge": "us.anthropic.claude-sonnet-4-20250514-v1:0",
      "judged-dtm":   
    }
  ]
}
```
 - Stores json records back to S3


In [1]:
# Import required libraries
import boto3  # AWS SDK for Python
import datetime
import json   # JSON handling
import time   # For rate limiting between API calls
from typing import List, Dict  # Type hints

# ============================================================================
# CONFIGURATION SECTION - Update these values
# ============================================================================
# S3 Configuration
INPUT_BUCKET = '183023889407-us-east-1-compliance-rule-generator'
INPUT_PREFIX = 'scenarios/'  # Folder path in S3 where scenarios are stored
OUTPUT_BUCKET = '183023889407-us-east-1-compliance-rule-generator'
OUTPUT_PREFIX = 'scenarios-judged/'  # Folder path for results
# AWS Region
AWS_REGION = 'us-east-1'
# AWS Bedrock Knowledge Base containing NIST policies
KNOWLEDGE_BASE_ID = 'T8EW10IU3Z'

MAX_TOKENS = 4096
TEMPERATURE = 0.7

# Available Bedrock model ARNs with performance notes
MODELS = {
    'premium': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/global.anthropic.claude-opus-4-5-20251101-v1:0', # not available
    'good': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/global.anthropic.claude-sonnet-4-5-20250929-v1:0', # times out
    'balanced': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.anthropic.claude-sonnet-4-20250514-v1:0',  # recommended
    'fast_cheap': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.anthropic.claude-haiku-4-5-20251001-v1:0',
    'aws_native_premier': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-premier-v1:0',
    'aws_native_pro': 'arn:aws:bedrock:us-east-1:183023889407:inference-profile/us.amazon.nova-pro-v1:0'
}
MODEL_ARN = MODELS['balanced']  # Default model selection

# JSON tool configuration for Bedrock Converse API
# Forces the model to return structured JSON with specific schema
TOOL_CONFIG = {
    "tools": [{
        "toolSpec": {
            "name": "judged_scenario_json",
            "description": "Return judged compliance scenarios as JSON",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "scenarios": {  # Array of scenario objects
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "judged-compliant": {"type": "boolean"},
                                    "judged-compliant-reason": {"type": "string"}
                                },
                                "required": ["judged-compliant", "judged-compliant-reason"]
                            }
                        }
                    },
                    "required": ["scenarios"]
                }
            }
        }
    }],
    "toolChoice": {"tool": {"name": "judged_scenario_json"}}  # Force use of the JSON tool
}

# Initialize AWS Bedrock clients
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name='us-east-1')  # For knowledge base retrieval
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')  # For model inference

In [2]:
def load_scenarios_from_s3(input_bucket: str = INPUT_BUCKET, input_prefix: str = INPUT_PREFIX, object_name: str = "scenarios.json") -> List[Dict]:
    """
    Load scenarios from S3 JSON file.
    """
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket=input_bucket, Key=input_prefix+object_name)
    json_data = json.loads(response['Body'].read().decode('utf-8'))
    return json_data["scenarios"]


In [3]:
def save_scenarios_to_s3(scenarios: List[Dict], output_bucket: str = OUTPUT_BUCKET, output_prefix: str = OUTPUT_PREFIX, object_name: str = "scenarios.json"):
    """
    Save generated scenarios to a S3.
    """
    s3 = boto3.client('s3')
    json_data = json.dumps({"scenarios": scenarios}, indent=2)
    s3.put_object(Bucket=output_bucket, Key=output_prefix+object_name, Body=json_data)

In [4]:
def save_scenarios_to_file(scenarios: List[Dict], output_path: str):
    
    # Print scenarios to console for immediate review
    print(json.dumps(scenarios, indent=2))
    
    # Save to file with metadata and statistics
    with open(output_path, 'w') as f:
        json.dump({
            'total_scenarios': len(scenarios),
            'compliant_count': sum(1 for s in scenarios if s['is-compliant']),
            'non_compliant_count': sum(1 for s in scenarios if not s['is-compliant']),
            'judged compliant_count': sum(1 for s in scenarios if s['judged-compliant']),
            'judged non_compliant_count': sum(1 for s in scenarios if not s['judged-compliant']),
            'scenarios': scenarios
        }, f, indent=2)

In [17]:
def judge_scenarios(source_scenarios: List[Dict], model_arn: str, kb_id: str = KNOWLEDGE_BASE_ID) -> List[Dict]:
    """
    Process scenarios and add judgment fields.
    """

    # Extract model ID from ARN (Converse API requires model ID, not full ARN)
    model_id = model_arn.split('/')[-1] if '/' in model_arn else model_arn
    
    judged_scenarios = []
    for scenario in source_scenarios:
        judged_scenario = scenario.copy()
        
        prompt = f"""
        You are **ComplianceEvaluator**, an expert AI compliance analyst specializing in NIST 800-53 security and privacy controls. 
        Your mission is to evaluate organizational scenarios, systems, and practices against NIST 800-53 controls and organizational 
        policies stored in your knowledge base.
        
        **Your Expertise Includes:**
        - Deep understanding of all NIST 800-53 Rev. 5 control families (AC, AT, AU, CA, CM, CP, IA, IR, MA, MP, PE, PL, PM, PS, PT, RA, SA, SC, SI, SR)
        - Control baseline mapping (Low, Moderate, High impact levels)
        - FedRAMP, FISMA, and related compliance frameworks
        - Risk assessment methodologies
        - Security architecture evaluation
        - Policy-to-control mapping
        
        **Your Disposition:**
        - Thorough and methodical in analysis
        - Evidence-focused in assessments
        
        Knowledgebase Retrieval Strategy
        
        When evaluating any scenario, you **MUST** search for **all policies referenced in the scenario**, 
        which are listed in the scenario.  For example, Policies referenced: CA-1.a.1.(a), RA-1.a.1, CM-1.a.1, PS-1.a.1

        Respond with JSON format:
        {{
          "judged-compliant": true/false, true if you determined the scenario is compliant with the organizational 
        policies stored in your knowledge base.  false if the scenario is not compliant.
          "judged-compliant-reason": "Empty if compliant. If the scenario is not compliant, explain very briefly why it is not compliant, citing
          exactly the policy ID(s) is violates."
        }}

        **Here is the compliance scenario to evaluate**:
        {scenario["scenario-detail"]}
        """

        response = bedrock_agent_runtime.retrieve_and_generate(
            input={'text': prompt},
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': model_arn,
                    'generationConfiguration': {
                        'inferenceConfig': {
                            'textInferenceConfig': {
                                'maxTokens': MAX_TOKENS,
                                'temperature': TEMPERATURE
                            }
                        }
                    }
                }
            }
        )
            
        response_text = response['output']['text']
        try:
            if '{' in response_text:
                json_start = response_text.find('{')
                json_end = response_text.rfind('}') + 1
                json_str = response_text[json_start:json_end]
                result = json.loads(json_str)
                judged_scenario["judged-compliant"] = result.get('judged-compliant')
                judged_scenario["judged-compliant-reason"] = result.get('judged-compliant-reason', '')
            else:
                judged_scenario["judged-compliant"] = 'compliant' in response_text.lower()
                judged_scenario["judged-compliant-reason"] = response_text
        except:
            judged_scenario["judged-compliant"] = None
            judged_scenario["judged-compliant-reason"] = response_text
        judged_scenario["llm-judge"] = MODEL_ARN.split('/')[-1]
        judged_scenario["judged-dtm"] = datetime.datetime.now().isoformat()
        judged_scenarios.append(judged_scenario)
    
    return judged_scenarios

In [18]:
source_scenarios_file = "scenarios.json"
judged_scenarios_file = "judged_scenarios.json"

source_scenarios = load_scenarios_from_s3(INPUT_BUCKET, INPUT_PREFIX, source_scenarios_file)

judged_scenarios = judge_scenarios(
    source_scenarios,
    MODEL_ARN,
    KNOWLEDGE_BASE_ID
)
save_scenarios_to_file(judged_scenarios, '/home/sagemaker-user/' + judged_scenarios_file)
save_scenarios_to_s3(judged_scenarios, OUTPUT_BUCKET, OUTPUT_PREFIX, judged_scenarios_file)



[
  {
    "scenario-id": "scenario-id-1",
    "scenario-detail": "MegaTech Solutions, a multinational technology corporation with 25,000+ employees across 60+ countries, is implementing a comprehensive cybersecurity governance program for their cloud-based enterprise resource planning (ERP) system processing financial data under SOX regulations. The Chief Information Security Officer (CISO) developed and disseminated a comprehensive Assessment, Authorization, and Monitoring (AAM) policy per CA-1.a.1.(a) addressing purpose, scope, roles, responsibilities, management commitment, coordination among organizational entities, and compliance requirements across all business units within the 10,000+ employee organization. The Chief Risk Officer (CRO) established a three-tier risk assessment policy framework per RA-1.a.1 covering organization-level, business unit-level, and system-level assessments with quarterly reviews and annual comprehensive evaluations. The CISO implemented configuration m

In [16]:
# Quick diagnostic - run this cell
try:
    # Test 1: Check KB status
    bedrock_agent = boto3.client('bedrock-agent', region_name='us-east-1')
    kb_info = bedrock_agent.get_knowledge_base(knowledgeBaseId=KNOWLEDGE_BASE_ID)
    print(f"KB Status: {kb_info['knowledgeBase']['status']}")
    
    # Test 2: Try simple retrieval
    response = bedrock_agent_runtime.retrieve(
        knowledgeBaseId=KNOWLEDGE_BASE_ID,
        retrievalQuery={'text': 'NIST access control'}
    )
    print(f"Retrieval works: {len(response['retrievalResults'])} results")
    
    # Test 3: Try different model
    response = bedrock_agent_runtime.retrieve_and_generate(
        input={'text': 'Test query'},
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': KNOWLEDGE_BASE_ID,
                'modelArn': MODELS['fast_cheap']  # Try Haiku instead
            }
        }
    )
    print("Haiku model works!")
    
except Exception as e:
    print(f"Error: {e}")
    print("Try: MODEL_ARN = MODELS['fast_cheap']")


KB Status: ACTIVE
Retrieval works: 5 results
Haiku model works!
